<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/Turku-MeteoSat_historical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.7/691.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.12.19
    Uninstalling flatbuffers-25.12.19:
      Successfully uninstalled flatbuffers-25.12.19
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 4.2 MB/s eta 0:00:00


In [4]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 60.4518,
	"longitude": 22.2666,
	"start_date": "2024-01-01",
	"end_date": "2025-01-01",
	"hourly": "wind_speed_10m",
	"wind_speed_unit": "ms",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_wind_speed_10m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 60.448883056640625°N 22.257400512695312°E
Elevation: 19.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                           date  wind_speed_10m
0    2024-01-01 00:00:00+00:00             3.3
1    2024-01-01 01:00:00+00:00             3.2
2    2024-01-01 02:00:00+00:00             3.4
3    2024-01-01 03:00:00+00:00             3.5
4    2024-01-01 04:00:00+00:00             3.8
...                        ...             ...
8803 2025-01-01 19:00:00+00:00             3.6
8804 2025-01-01 20:00:00+00:00             3.6
8805 2025-01-01 21:00:00+00:00             4.1
8806 2025-01-01 22:00:00+00:00             3.4
8807 2025-01-01 23:00:00+00:00             3.4

[8808 rows x 2 columns]


In [5]:
hourly_dataframe['date_only'] = hourly_dataframe['date'].dt.date
hourly_dataframe['hour_only'] = hourly_dataframe['date'].dt.hour

# Display the updated DataFrame with new date and hour columns
display(hourly_dataframe.head())

,date,wind_speed_10m,date_only,hour_only
0,2024-01-01 00:00:00+00:00,3.3,2024-01-01,0
1,2024-01-01 01:00:00+00:00,3.2,2024-01-01,1
2,2024-01-01 02:00:00+00:00,3.4,2024-01-01,2
3,2024-01-01 03:00:00+00:00,3.5,2024-01-01,3
4,2024-01-01 04:00:00+00:00,3.8,2024-01-01,4


In [8]:
# Save the DataFrame to an Excel file
excel_file_path = 'wind_speed_data.xlsx'

# Remove timezone information from the 'date' column before saving to Excel
hourly_dataframe['date'] = hourly_dataframe['date'].dt.tz_localize(None)

hourly_dataframe.to_excel(excel_file_path, index=False)

print(f"DataFrame successfully saved to {excel_file_path}")

DataFrame successfully saved to wind_speed_data.xlsx
